# Series 8: Neural Networks feat. TensorFlow

## Problem 2 (Convolutional Neural Networks)

This notebook simply implements the simple version of the TensorFlow MNIST digit recognition tutorial, located at [www.tensorflow.org/versions/r0.8/tutorials/mnist/beginners/index.html][0].

This implementation simply uses softmax regression, i.e. nonbinary logistic regression.

[0]: https://www.tensorflow.org/versions/r0.8/tutorials/mnist/beginners/index.html

In [4]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/MNIST_data/", one_hot=True)

Extracting data/MNIST_data/train-images-idx3-ubyte.gz
Extracting data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting data/MNIST_data/t10k-labels-idx1-ubyte.gz
